In [0]:
import numpy as np
import pandas as pd
import tensorflow as tf

In [41]:
# Connecting to google drive
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Specifying the location of the dataset and extracting it using pandas read_csv functionality
train_dataset = pd.read_csv("gdrive/My Drive/dataset/fashion_mnist_train.csv")
test_dataset = pd.read_csv("gdrive/My Drive/dataset/fashion_mnist_test.csv")

In [43]:
print(f"Train dataset dimensions : {train_dataset.shape}")
print(f"Test dataset dimensions : {test_dataset.shape}")

Train dataset dimensions : (60000, 785)
Test dataset dimensions : (10000, 785)


In [0]:
# Dropping the label column from test and train datasets and forming x_train and x_test respectively
# Putting the labels into a separate y_train and y_test dataframe.
x_train = train_dataset.drop(columns=["label"])
y_train = train_dataset["label"]

In [0]:
x_test = test_dataset.drop(columns=["label"])
y_test = test_dataset["label"]

In [46]:
# Normalizing the train dataset by setting the value of pixel colors in the range 0 to 1
x_train = x_train.to_numpy()/255.0
print(f"Train dataset dimensions : {x_train.shape}")

x_test = x_test.to_numpy()/255.0
print(f"Test dataset dimensions : {x_test.shape}")

Train dataset dimensions : (60000, 784)
Test dataset dimensions : (10000, 784)


In [47]:
# Converting labels pandas dataframe to numpy arrays
y_train = y_train.to_numpy()
print(f"Train dataset labels dimensions : {y_train.shape}")

y_test = y_test.to_numpy()
print(f"Test dataset labels dimensions : {y_test.shape}")

Train dataset labels dimensions : (60000,)
Test dataset labels dimensions : (10000,)


In [48]:
# Reshaping the test and train data to a 4D matrix : [instances, 28, 28, 1] 
x_train = np.reshape(x_train, (-1, 28, 28, 1)).astype('float32')

x_test = np.reshape(x_test, (-1, 28, 28, 1)).astype('float32')

# one hot encoded test and train labels:
y_train = tf.keras.utils.to_categorical(y_train.astype('float32'))

y_test = tf.keras.utils.to_categorical(y_test.astype('float32'))

print(f"Train dataset dimensions : {x_train.shape}")
print(f"Train dataset labels dimensions : {y_train.shape}")
print(f"Test dataset dimensions : {x_test.shape}")
print(f"Test dataset labels dimensions : {y_test.shape}")

Train dataset dimensions : (60000, 28, 28, 1)
Train dataset labels dimensions : (60000, 10)
Test dataset dimensions : (10000, 28, 28, 1)
Test dataset labels dimensions : (10000, 10)


In [0]:
class Length(tf.keras.layers.Layer):
    def call(self, inputs, **kwargs):
        return tf.keras.backend.sqrt(tf.keras.backend.sum(tf.keras.backend.square(inputs), -1))

    def compute_output_shape(self, input_shape):
        return input_shape[:-1]

In [0]:
class Mask(tf.keras.layers.Layer):

    def call(self, inputs, **kwargs):
        
        if type(inputs) is list:  # true label is provided with shape = [batch_size, n_classes], i.e. one-hot code.
            assert len(inputs) == 2
            inputs, mask = inputs
        else:  # if no true label, mask by the max length of vectors of capsules
            x = inputs
            # Enlarge the range of values in x to make max(new_x)=1 and others < 0
            x = (x - tf.keras.backend.max(x, 1, True)) / tf.keras.backend.epsilon() + 1
            mask = tf.keras.backend.clip(x, 0, 1)  # the max value in x clipped to 1 and other to 0

        # masked inputs, shape = [batch_size, dim_vector]
        inputs_masked = tf.keras.backend.batch_dot(inputs, mask, [1, 1])
        return inputs_masked

    def compute_output_shape(self, input_shape):
        if type(input_shape[0]) is tuple:  # true label provided
            return tuple([None, input_shape[0][-1]])
        else:
            return tuple([None, input_shape[-1]])


In [0]:
def squash_function(x, axis=-1):
    s_squared_norm = tf.keras.backend.sum(tf.keras.backend.square(x), axis, keepdims=True)
    scale = s_squared_norm / (1 + s_squared_norm) / tf.keras.backend.sqrt(s_squared_norm)
    return scale * x

In [0]:
class Capsule(tf.keras.layers.Layer):
    def __init__(self, num_capsule, dimension_vector, num_routing=3, kernel_initializer='glorot_uniform',
                bias_initializer='zeros', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule 
        self.dimension_vector = dimension_vector
        self.num_routing = num_routing
        self.kernel_initializer = tf.keras.initializers.get(kernel_initializer)
        self.bias_initializer = tf.keras.initializers.get(bias_initializer)
        
    def build(self, input_shape):
        assert len(input_shape) >= 3
        self.input_num_capsule = int(input_shape[1])
        self.input_dim_vector = int(input_shape[2])
        self.W = self.add_weight(shape=[self.input_num_capsule, self.num_capsule, self.input_dim_vector, self.dimension_vector], 
                                initializer=self.kernel_initializer, name='W')
        
        self.bias = self.add_weight(shape=[1, self.input_num_capsule, self.num_capsule, 1, 1], 
                                   initializer=self.bias_initializer, name='bias', trainable=False)
        
        self.built = True
        
    def call(self, inputs, training=None):
        inputs_expand = tf.keras.backend.expand_dims(tf.keras.backend.expand_dims(inputs, 2), 2)
        
        inputs_tiled = tf.keras.backend.tile(inputs_expand, [1, 1, self.num_capsule, 1 , 1])
        
        inputs_hat = tf.scan(lambda ac, x:tf.keras.backend.batch_dot(x, self.W, [3, 2]), 
                            elems=inputs_tiled, initializer=tf.keras.backend.zeros([self.input_num_capsule, self.num_capsule, 1, self.dimension_vector]))
        
        assert self.num_routing > 0, 'the num_routing should be > 0.'
        
        for i in range(self.num_routing):
            c = tf.nn.softmax(self.bias, dim=2)
            
            outputs = squash_function(tf.keras.backend.sum(c * inputs_hat, 1, keepdims=True))
            
            if i != self.num_routing - 1:
                self.bias = self.bias + tf.keras.backend.sum(inputs_hat * outputs, -1, keepdims=True)
        
        return tf.keras.backend.reshape(outputs, [-1, self.num_capsule, self.dimension_vector])
    
    def compute_output_shape(self, input_shape):
        return tuple([None, self.num_capsule, self.dimension_vector]) 

In [0]:
def PrimaryCap(inputs, dimension_vector, num_channels, kernel_size, strides, padding):
    output = tf.keras.layers.Conv2D(filters=dimension_vector*num_channels, kernel_size=kernel_size, strides=strides, padding=padding)(inputs)
    outputs = tf.keras.layers.Reshape(target_shape=[1152, dimension_vector])(output)
    return tf.keras.layers.Lambda(squash_function)(outputs)

In [0]:
def CapsNet(input_shape, n_class, num_routing):
    
    x = tf.keras.layers.Input(shape=input_shape)

    # Convolutional layer 1:
    conv1 = tf.keras.layers.Conv2D(filters=256, kernel_size=9, strides=1, padding='valid', activation='relu', name='conv1')(x)
    # Convolutional layer with squash activation function which is later on reshape to [None, num_capsule, dimension_vector]
    primarycaps = PrimaryCap(conv1, dimension_vector=8, num_channels=32, kernel_size=9, strides=2, padding='valid')
    # Capsule layer: Contains the routing algorithm (routing by agreement).
    digitcaps = Capsule(num_capsule=n_class, dimension_vector=16, num_routing=num_routing, name='digitcaps')(primarycaps)

    # Layer 4: This is an auxiliary layer to replace each capsule with its length. Just to match the true label's shape.
    # If using tensorflow, this will not be necessary. :)
    out_caps = Length(name='out_caps')(digitcaps)

    # Decoder network.
    y = tf.keras.layers.Input(shape=(n_class,))
    masked = Mask()([digitcaps, y])  # The true label is used to mask the output of capsule layer.
    x_recon = tf.keras.layers.Dense(512, activation='relu')(masked)
    x_recon = tf.keras.layers.Dense(1024, activation='relu')(x_recon)
    x_recon = tf.keras.layers.Dense(784, activation='sigmoid')(x_recon)
    x_recon = tf.keras.layers.Reshape(target_shape=[28, 28, 1], name='out_recon')(x_recon)

    # two-input-two-output keras Model
    return tf.keras.models.Model([x, y], [out_caps, x_recon])

In [0]:
def margin_loss(y_true, y_pred):
    L = y_true * tf.keras.backend.square(tf.keras.backend.maximum(0., 0.9 - y_pred)) + \
        0.5 * (1 - y_true) * tf.keras.backend.square(tf.keras.backend.maximum(0., y_pred - 0.1))

    return tf.keras.backend.mean(tf.keras.backend.sum(L, 1))

In [56]:
model = CapsNet(input_shape=[28, 28, 1],
                n_class=10,
                num_routing=3)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 20, 20, 256)  20992       input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 6, 6, 256)    5308672     conv1[0][0]                      
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 1152, 8)      0           conv2d_2[0][0]                   
____________________________________________________________________________________________

In [57]:
# compile the model
model.compile(optimizer='adam',
              loss=[margin_loss, 'mse'],
              loss_weights=[1., 0.0005],
              metrics={'out_caps': 'accuracy'})

model.fit(x=[x_train, y_train], y=[y_train, x_train], validation_data=[[x_test, y_test], [y_test, x_test]], epochs=1)


Train on 60000 samples, validate on 10000 samples
60000/60000 [==============================] - 300s 5ms/sample - loss: 0.1359 - out_caps_loss: 0.1358 - out_recon_loss: 0.0843 - out_caps_acc: 0.8193 - val_loss: 0.0962 - val_out_caps_loss: 0.0961 - val_out_recon_loss: 0.0556 - val_out_caps_acc: 0.8718
